In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql import types

In [3]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("homework") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/06 15:10:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Q1

In [3]:
pyspark.__version__

'3.3.2'

In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-06 14:18:06--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 108.138.245.58, 108.138.245.96, 108.138.245.225, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|108.138.245.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  21.5MB/s    in 2.9s    

2025-03-06 14:18:09 (21.5 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [4]:
year=2024
month=10
input_path = f'yellow_tripdata_2024-10.parquet'

In [6]:
output_path = f'data/pq/yellow/{year}/{month:02d}/'
df_yellow = spark.read \
        .option("header", "true") \
        .parquet(input_path)

df_yellow.repartition(4).write.parquet(output_path, mode='overwrite')

## Q2

In [11]:
!ls -lh /home/tsedo/notebooks/data/pq/yellow/2024/10

total 97M
-rw-r--r-- 1 tsedo tsedo   0 Mar  6 14:23 _SUCCESS
-rw-r--r-- 1 tsedo tsedo 25M Mar  6 14:23 part-00000-abb9be01-b172-4e0d-9c22-71b114f04d54-c000.snappy.parquet
-rw-r--r-- 1 tsedo tsedo 25M Mar  6 14:23 part-00001-abb9be01-b172-4e0d-9c22-71b114f04d54-c000.snappy.parquet
-rw-r--r-- 1 tsedo tsedo 25M Mar  6 14:23 part-00002-abb9be01-b172-4e0d-9c22-71b114f04d54-c000.snappy.parquet
-rw-r--r-- 1 tsedo tsedo 25M Mar  6 14:23 part-00003-abb9be01-b172-4e0d-9c22-71b114f04d54-c000.snappy.parquet


## Q3

In [ ]:
df_yellow.registerTempTable('yellow_data')

In [23]:
spark.sql("""
SELECT count(1)
    FROM yellow_data 
    where day(tpep_dropoff_datetime) = 15
    
""").show()

+--------+
|count(1)|
+--------+
|  128811|
+--------+



## Q4

In [36]:
spark.sql("""
    SELECT MAX(UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 3600 AS max_hours
FROM yellow_data;

""").show(4)

+------------------+
|         max_hours|
+------------------+
|162.61777777777777|
+------------------+



## Q6

In [19]:
df_zones = spark.read \
        .option("header", "true") \
        .csv("taxi_zone_lookup.csv")

In [11]:
df_join = df_yellow.join(df_zones, df_yellow.PULocationID == df_zones.LocationID, 'left')


In [16]:
df_join.registerTempTable('df_join')

In [18]:
spark.sql("""
SELECT Zone, count(1) as cnt
FROM df_join
group by zone
order by cnt
;
    
""").show(1)

+--------------------+---+
|                Zone|cnt|
+--------------------+---+
|Governor's Island...|  1|
+--------------------+---+
only showing top 1 row

